In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from feature_selector import FeatureSelector #credits to Will Koehrsen
#his class is on github https://github.com/WillKoehrsen/feature-selector/blob/master/Feature%20Selector%20Usage.ipynb

In [4]:
#Read the dataset
data_full = pd.read_csv("CE802_Ass_2019_Data.csv")
data_class = data_full["Class"]
data_full = data_full.drop(columns = ["Class"],axis = 1)


In [ ]:
fs = FeatureSelector(data = data_full, labels = data_class)

In [109]:
#dictionary with the parameters to loop for
parameters_features = {'missing_threshold': np.arange(0.35,0.45,0.05),    
                       'correlation_threshold': np.arange(0.35,0.75,0.05),
                       'task': 'classification',    
                       'eval_metric': 'auc', 
                       'cumulative_importance': np.arange(0.65,1,0.05)}



In [110]:
#the following for with search for the best score based on a svm classifier
#and with the best score, it will find the best parameters to do feature selection
score = 0.0000000
for idx,x in enumerate(parameters_features['missing_threshold']):
    for idy,y in enumerate(parameters_features['correlation_threshold']):
        for idz,z in enumerate(parameters_features['cumulative_importance']):
            fs.identify_all(selection_params = {'missing_threshold': parameters_features['missing_threshold'][idx], 
                                                'correlation_threshold': parameters_features['correlation_threshold'][idy], 
                                                'task': 'classification', 'eval_metric': 'auc', 
                                                 'cumulative_importance': parameters_features['cumulative_importance'][idz]})
            train_removed_all_once = fs.remove(methods = 'all', keep_one_hot = True)
            
            data_features= train_removed_all_once.to_numpy()
            
            imputer = KNNImputer(n_neighbors=2, weights="uniform")
            data_features = imputer.fit_transform(data_features)
            data_feat = pd.DataFrame(data_features, index=range(data_features.shape[0]),
                                      columns=range(data_features.shape[1]))

            #Lets first normalize the features for K-NN and SVM
            scaler = StandardScaler()
            scaler.fit(data_feat)
            data_feat = scaler.transform(data_feat)

            #Splitting of data to see model accuracy after cross validation and gridsearch
            data_feat_train, data_feat_test, data_class_train, data_class_test = train_test_split(data_feat,data_class,test_size=0.25,stratify=data_class,random_state=1234)
            
            clf_svm = svm.SVC()
            param_grid = {'C': np.logspace(-1, 3, 9),  
                          'gamma': np.logspace(-7, -0, 8)}

            svm_gridsearch = GridSearchCV(clf_svm,param_grid,n_jobs=-1, cv = 10)
            svm_gridsearch.fit(data_feat_train,data_class_train)
            if svm_gridsearch.best_score_ >= score:
                score = svm_gridsearch.best_score_
                parameters_values = [parameters_features['missing_threshold'][idx],parameters_features['correlation_threshold'][idy],parameters_features['cumulative_importance'][idz]]
            #print("Best parameters: " + str(svm_gridsearch.best_params_))
            #print("Best score : " + str(svm_gridsearch.best_score_))

0 features with greater than 0.35 missing values.

0 features with a single unique value.

4 features with a correlation magnitude greater than 0.35.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's auc: 0.668831	valid_0's binary_logloss: 0.640707
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.744228	valid_0's binary_logloss: 0.676268
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.764069	valid_0's binary_logloss: 0.657408
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	valid_0's auc: 0.61039	valid_0's binary_logloss: 0.672997
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[177]	valid_0's auc: 0.775613	valid_0's binary_logloss: 0.586718
Training unt

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[23]	valid_0's auc: 0.626263	valid_0's binary_logloss: 0.660929

0 features with zero importance after one-hot encoding.

12 features required for cumulative importance of 0.80 after one hot encoding.
8 features do not contribute to cumulative importance of 0.80.

12 total features out of 20 identified for removal after one-hot encoding.

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance'] methods have been run

Removed 12 features.
0 features with greater than 0.35 missing values.

0 features with a single unique value.

4 features with a correlation magnitude greater than 0.35.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[81]	valid_0's auc: 0.761183	valid_0's binary_logloss: 0.565086
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration 

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.662338	valid_0's binary_logloss: 0.652834
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[109]	valid_0's auc: 0.752525	valid_0's binary_logloss: 0.584608
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[119]	valid_0's auc: 0.816017	valid_0's binary_logloss: 0.541659
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[22]	valid_0's auc: 0.587302	valid_0's binary_logloss: 0.683151

0 features with zero importance after one-hot encoding.

9 features required for cumulative importance of 0.65 after one hot encoding.
11 features do not contribute to cumulative importance of 0.65.

13 total features out of 20 identified for removal after one-hot encoding.

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance'

Early stopping, best iteration is:
[150]	valid_0's auc: 0.712121	valid_0's binary_logloss: 0.619626
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[54]	valid_0's auc: 0.670274	valid_0's binary_logloss: 0.660278
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[180]	valid_0's auc: 0.735931	valid_0's binary_logloss: 0.608117
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	valid_0's auc: 0.702742	valid_0's binary_logloss: 0.618625
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.606782	valid_0's binary_logloss: 0.680599
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's auc: 0.71645	valid_0's binary_logloss: 0.651486
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:

0 features with greater than 0.35 missing values.

0 features with a single unique value.

3 features with a correlation magnitude greater than 0.45.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[63]	valid_0's auc: 0.694084	valid_0's binary_logloss: 0.630458
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[111]	valid_0's auc: 0.743867	valid_0's binary_logloss: 0.615003
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[32]	valid_0's auc: 0.704185	valid_0's binary_logloss: 0.634622
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[203]	valid_0's auc: 0.841991	valid_0's binary_logloss: 0.507523
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.728355	valid_0's binary_logloss: 0.676846
Training 

...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[186]	valid_0's auc: 0.793651	valid_0's binary_logloss: 0.56

0 features with zero importance after one-hot encoding.

14 features required for cumulative importance of 0.85 after one hot encoding.
6 features do not contribute to cumulative importance of 0.85.

7 total features out of 20 identified for removal after one-hot encoding.

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance'] methods have been run

Removed 7 features.
0 features with greater than 0.45 missing values.

0 features with a single unique value.

1 features with a correlation magnitude greater than 0.60.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[160]	valid_0's auc: 0.751804	valid_0's binary_logloss: 0.605608
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.515873	valid_0's binary_logloss: 0.685652
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[111]	valid_0's auc: 0.735209	valid_0's binary_logloss: 0.620855
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	valid_0's auc: 0.704906	valid_0's binary_logloss: 0.618142
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[25]	valid_0's auc: 0.697691	valid_0's binary_logloss: 0.633292

0 features with zero importance after one-hot encoding.

10 features required for cumulative importance of 0.70 after one hot encoding.
10 features do not contribute to cumulative importance of 0.70.

11 total features out of 20 identified for removal after one-hot encoding.

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance']

[123]	valid_0's auc: 0.720779	valid_0's binary_logloss: 0.615895
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	valid_0's auc: 0.580808	valid_0's binary_logloss: 0.679801
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.69228	valid_0's binary_logloss: 0.676813
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[19]	valid_0's auc: 0.701299	valid_0's binary_logloss: 0.631326
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's auc: 0.606782	valid_0's binary_logloss: 0.676429
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	valid_0's auc: 0.734488	valid_0's binary_logloss: 0.614024
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's auc: 0.750361	valid_0

Early stopping, best iteration is:
[124]	valid_0's auc: 0.755411	valid_0's binary_logloss: 0.581206
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[71]	valid_0's auc: 0.726551	valid_0's binary_logloss: 0.621037
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[142]	valid_0's auc: 0.771284	valid_0's binary_logloss: 0.569505
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[27]	valid_0's auc: 0.694805	valid_0's binary_logloss: 0.633353
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's auc: 0.695887	valid_0's binary_logloss: 0.669964
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.784993	valid_0's binary_logloss: 0.618134
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is

0 features with greater than 0.45 missing values.

0 features with a single unique value.

1 features with a correlation magnitude greater than 0.70.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.576479	valid_0's binary_logloss: 0.675798
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[37]	valid_0's auc: 0.790043	valid_0's binary_logloss: 0.583154
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	valid_0's auc: 0.684704	valid_0's binary_logloss: 0.628785
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's auc: 0.707792	valid_0's binary_logloss: 0.623362
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[218]	valid_0's auc: 0.781385	valid_0's binary_logloss: 0.570993
Training 

In [111]:
#"0.7356330014224751{'C': 3.1622776601683795, 'gamma': 0.1}"
print(score)
print(parameters_values)
#[0.39999999999999997, 0.7, 0.7500000000000001]

0.738122332859175
[0.39999999999999997, 0.7, 0.7500000000000001]


In [14]:
fs = FeatureSelector(data = data_full, labels = data_class)
fs.identify_all(selection_params = {'missing_threshold': 0.39999999999999997,    
                                    'correlation_threshold': 0.7, 
                                    'task': 'classification',    
                                    'eval_metric': 'auc', 
                                    'cumulative_importance': 0.7500000000000001})


train_removed_all_once = fs.remove(methods = 'all', keep_one_hot = True)
print(train_removed_all_once)           
data_features= train_removed_all_once.to_numpy()
          
imputer = KNNImputer(n_neighbors=2, weights="uniform")
data_features = imputer.fit_transform(data_features)
data_feat = pd.DataFrame(data_features, index=range(data_features.shape[0]),
                                      columns=range(data_features.shape[1])) 
#Lets first normalize the features for K-NN and SVM
scaler = StandardScaler()
scaler.fit(data_feat)
data_feat = scaler.transform(data_feat)

#Splitting of data to see model accuracy after cross validation and gridsearch
data_feat_train, data_feat_test, data_class_train, data_class_test = train_test_split(data_feat,data_class,test_size=0.25,stratify=data_class,random_state=1234)
            
clf_svm = svm.SVC()
param_grid = {'C': np.logspace(-1, 3, 9),  
              'gamma': np.logspace(-7, -0, 8)}

svm_gridsearch = GridSearchCV(clf_svm,param_grid,n_jobs=-1, cv = 10)
svm_gridsearch.fit(data_feat_train,data_class_train)
print("Best parameters: " + str(svm_gridsearch.best_params_))
print("Best score : " + str(svm_gridsearch.best_score_))
#Now lets used cross validation in the whole data set, but with the best parameters by gridsearch
svm_model = svm.SVC(C = svm_gridsearch.best_params_['C'],gamma=svm_gridsearch.best_params_['gamma'])

#Now lets used cross validation in the whole data set, but with the best parameters by gridsearch
score_svm = cross_val_score(svm_model,data_feat_train,data_class_train,cv=10,n_jobs=-1)
print('Average accuracy:', np.mean(score_svm))

#Now lets compute the confussion matrix by splitting the data into trainning and testing
svm_model.fit(data_feat_train,data_class_train)
svm_pred = svm_model.predict(data_feat_test)
print(confusion_matrix(data_class_test, svm_pred))
print(classification_report(data_class_test, svm_pred))

0 features with greater than 0.40 missing values.

0 features with a single unique value.

1 features with a correlation magnitude greater than 0.70.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.70202	valid_0's binary_logloss: 0.636678
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's auc: 0.722944	valid_0's binary_logloss: 0.617109
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[119]	valid_0's auc: 0.723665	valid_0's binary_logloss: 0.614158
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.345599	valid_0's binary_logloss: 0.692999
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[222]	valid_0's auc: 0.779221	valid_0's binary_logloss: 0.564797
Training u

In [8]:
knn_gridcv = KNeighborsClassifier()
#create a dictionary with the number of neighbors to try
param_gridsearch = {'n_neighbors': np.arange(1,80)}

knn_gridsearch = GridSearchCV(knn_gridcv,param_gridsearch,cv=10)
knn_gridsearch.fit(data_feat_train,data_class_train)
print("Best parameters: " + str(knn_gridsearch.best_params_))
print("Best score: "+ str(knn_gridsearch.best_score_))

Best parameters: {'n_neighbors': 14}
Best score: 0.6561877667140825


In [9]:
#Now lets used cross validation in the whole data set, but with the best parameters by gridsearch
knn_model = KNeighborsClassifier(n_neighbors = knn_gridsearch.best_params_['n_neighbors'])

#Now lets used cross validation in the whole data set, but with the best parameters by gridsearch
score_knn = cross_val_score(knn_model,data_feat_train,data_class_train,cv=10,n_jobs=-1)
print('Average accuracy:', np.mean(score_knn))

#Now lets compute the confussion matrix by splitting the data into trainning and testing
knn_model.fit(data_feat_train,data_class_train)
knn_pred = knn_model.predict(data_feat_test)
print(confusion_matrix(data_class_test, knn_pred))
print(classification_report(data_class_test, knn_pred))

Average accuracy: 0.6561877667140825
[[61  9]
 [33 22]]
              precision    recall  f1-score   support

       False       0.65      0.87      0.74        70
        True       0.71      0.40      0.51        55

    accuracy                           0.66       125
   macro avg       0.68      0.64      0.63       125
weighted avg       0.68      0.66      0.64       125



In [10]:
#Decision tree using grid search
clf_tree = tree.DecisionTreeClassifier(criterion = 'entropy')
param_grid = {'max_depth': np.arange(4,21),'min_samples_split': np.arange(4,21),'min_samples_leaf': np.arange(4,21)}
tree_gridcv = GridSearchCV(clf_tree,param_grid,cv=10 ,n_jobs=-1)
tree_gridcv.fit(data_feat_train,data_class_train)

print("Best parameters: " + str(tree_gridcv.best_params_))
print("Best score: " + str(tree_gridcv.best_score_))

Best parameters: {'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 18}
Best score: 0.6104551920341394


In [11]:
#Now with these parameters, lets perform cross validation
clf_tree_prunned = tree.DecisionTreeClassifier(criterion = 'entropy',
                                               max_depth= tree_gridcv.best_params_['max_depth'],
                                               min_samples_leaf= tree_gridcv.best_params_['min_samples_leaf'],
                                               min_samples_split=tree_gridcv.best_params_['min_samples_split'] )

#Now lets used cross validation in the whole data set, but with the best parameters by gridsearch
score_tree = cross_val_score(clf_tree_prunned,data_feat_train,data_class_train,cv=10,n_jobs=-1)
print('Average accuracy:', np.mean(score_tree))

#Now lets compute the confussion matrix by splitting the data into trainning and testing
clf_tree_prunned.fit(data_feat_train,data_class_train)
tree_pred = clf_tree_prunned.predict(data_feat_test)
print(confusion_matrix(data_class_test, tree_pred))
print(classification_report(data_class_test, tree_pred))

Average accuracy: 0.6104551920341394
[[36 34]
 [34 21]]
              precision    recall  f1-score   support

       False       0.51      0.51      0.51        70
        True       0.38      0.38      0.38        55

    accuracy                           0.46       125
   macro avg       0.45      0.45      0.45       125
weighted avg       0.46      0.46      0.46       125



XGBoost